In [45]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.dummy import DummyClassifier
import tensorflow as tf
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE

In [2]:
"""Tworzę nową rankę danych z różnicami między parami (różnica wieku, rasy, jak ważna dla ankietowanej osoby jest wspólna rasa,
różnice w ocenie oraz oczekiwaniach atrakcyjności, szczerości, inteligencji, dowcipu, ambicji, wspólnych zainteresowań,
orelacja wspólnych zainteresowań - dane możliwe do uzyskania podczas ankiety przed randką"""
df = pd.read_csv('speeddating.csv')
new_df = pd.DataFrame()
new_data_list = []
for row in df.itertuples():

    difference_age = row[6]
    difference_race = row[10]
    importance_same_race = row[11]
    difference_attractive_imporatance = abs(row[16]-row[40])
    difference_sincere_imporatance = abs(row[17]-row[41])
    difference_intelligence_imporatance = abs(row[18]-row[42])
    difference_funny_imporatance = abs(row[19]-row[43])
    difference_ambition_imporatance = abs(row[20]-row[44])
    difference_shared_interests_imporatance = abs(row[21]-row[45])
    attractive_difference = abs(row[28] - row[52])
    sinsere_difference = abs(row[29] - row[53])
    intelligence_difference = abs(row[30] - row[54])
    funny_difference = abs(row[31] - row[55])
    ambitous_difference = abs(row[32] - row[56])
    interests_correlation = row[108]
    target = row[123]

    new_data = {
        'difference_age': difference_age,
        'difference_race': difference_race,
        'importance_same_race': importance_same_race,
        'difference_attractive_imporatance': difference_attractive_imporatance,
        'difference_sincere_imporatance': difference_sincere_imporatance,
        'difference_intelligence_imporatance': difference_intelligence_imporatance,
        'difference_funny_imporatance': difference_funny_imporatance,
        'difference_ambition_imporatance': difference_ambition_imporatance,
        'difference_shared_interests_imporatance': difference_shared_interests_imporatance,
        'attractive_difference': attractive_difference,
        'sinsere_difference': sinsere_difference,
        'intelligence_difference': intelligence_difference,
        'funny_difference': funny_difference,
        'ambitous_difference': ambitous_difference,
        'interests_correlation': interests_correlation,
        'target': target
    }
    new_data_list.append(new_data)

new_df = pd.DataFrame(new_data_list)
new_df.to_csv('df_differences.csv', index=False)

In [ ]:
"""Sprawdzę jak wyglądają dane"""

In [3]:
new_df.head()
new_df.describe()
new_df.info()
new_df.isnull().sum()
new_df.dropna(inplace=True)
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8378 entries, 0 to 8377
Data columns (total 16 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   difference_age                           8378 non-null   float64
 1   difference_race                          8378 non-null   int64  
 2   importance_same_race                     8299 non-null   float64
 3   difference_attractive_imporatance        8210 non-null   float64
 4   difference_sincere_imporatance           8210 non-null   float64
 5   difference_intelligence_imporatance      8210 non-null   float64
 6   difference_funny_imporatance             8192 non-null   float64
 7   difference_ambition_imporatance          8174 non-null   float64
 8   difference_shared_interests_imporatance  8130 non-null   float64
 9   attractive_difference                    8069 non-null   float64
 10  sinsere_difference                       7997 no

In [ ]:
"""Tworzę pierwszy model - las losowy"""

In [4]:
X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
model = RandomForestClassifier(n_estimators=100, max_depth=10)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("accuracy: ", accuracy_score(y_test, y_pred))

accuracy:  0.843351548269581


In [ ]:
"""Dla podanego modelu accuracy wyniosło >0.80, sprawdzę macierz pomyłek"""

In [5]:
print(confusion_matrix(y_test, y_pred))

[[925   0]
 [172   1]]


In [16]:
"""Jak widać, wynik modelu jest "napompowany" wynikami negatywnymi, nie umie wskazać pozytywnych randek"""
"to pokazuje że accuracy NIE POWINNO być używane do oceny modeli - uczestnik nigdy nie dostanie propozycji randki"
"dla oceny poniższych modeli najważniejsza przyjętą miarą będzie stosunek w ilości sytuacji gdzie model nadał etykietę"
"1 dla danej testowej w której wynikiem faktycznie było 1 do ilości wszystkich danych w których model nadał etykietę 1"
"-chodzi o proponowanie ludziom w pierwszej kolejności takich randek które mają szanse powodzenia i by użytkownik jak najszybciej"
"znalazł odpowiednią osobę"

[[1098]]


In [6]:

for i in range(1,10):
    for j in range(1,10):
        # X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
        model = RandomForestClassifier(n_estimators=10*i, max_depth=10*j)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        print("_____________________")
        print("proba: ", i, " ", j)
        print("confusion matrix: ")
        print(confusion_matrix(y_test, y_pred))
        if confusion_matrix(y_test, y_pred)[1][1]+confusion_matrix(y_test, y_pred)[0][1] == 0:
            print("brak pozytywnych wyników")
        else:
            print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", confusion_matrix(y_test, y_pred)[1][1]/(confusion_matrix(y_test, y_pred)[1][1]+confusion_matrix(y_test, y_pred)[0][1]))
        print("_____________________")
        print("_____________________")

_____________________
proba:  1   1
confusion matrix: 
[[922   3]
 [169   4]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.5714285714285714
_____________________
_____________________
_____________________
proba:  1   2
confusion matrix: 
[[909  16]
 [161  12]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.42857142857142855
_____________________
_____________________
_____________________
proba:  1   3
confusion matrix: 
[[913  12]
 [160  13]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.52
_____________________
_____________________
_____________________
proba:  1   4
confusion matrix: 
[[907  18]
 [159  14]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.4375
_____________________
_____________________
_____________________
proba:  1   5
confusion matrix: 
[[912  13]
 [156  17]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.5666666666666667
_____________________
_____________________
_____

In [7]:
"wyniki są dobre dla parametrów n_estimators = 50 i max_depth = 90  , sprawdzę jeszcze kilka razy czy zawsze są one dobre czy tylko raz się udało dla takiego wylosowania danych"
model = RandomForestClassifier(n_estimators=50, max_depth=90)
for i in range(3):
    X_train, X_test, y_train, y_test = train_test_split(new_df.drop('target', axis=1), new_df['target'], test_size=0.15)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("_____________________")
    cm = confusion_matrix(y_test, y_pred)
    print("confusion matrix: ")
    print(cm)
    if cm[1][1]+cm[0][1] == 0:
        print("brak pozytywnych wyników")
    else:
        print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[918   2]
 [173   5]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.7142857142857143
_____________________
confusion matrix: 
[[885   3]
 [205   5]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.625
_____________________
confusion matrix: 
[[909   1]
 [178  10]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.9090909090909091


In [8]:
"Jako porównanie sprawdzimy jak poradzi sobie nadawanie etykiet losowo, proporcjonalnie do częstości występowania etykiet w danych"
"oraz jakie są szanse gdyby użytkownik miał pójść na randkę z jakąkolwiek osobą"
dummy = DummyClassifier(strategy='stratified')
dummy.fit(X_train, y_train)
y_pred = dummy.predict(X_test)
print("_____________________")
print("losowo ale z proporcjami")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
losowo ale z proporcjami
confusion matrix: 
[[765 145]
 [156  32]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.1807909604519774


In [ ]:
"Z jednej strony wyniki modelu sąznacznie lepsze, jednak mając zbiór ponad 1000 danych proponuje tylko około" \
"10 randek, co oznacza że aplikacja musiała by mieć bardzo dużą bazę użytkowników, żeby w ogóle działała"
"Teraz dla porównania spróbuję wytrenować sieć neuronową"

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["Precision"])

model.fit(X_train, y_train, epochs=5, batch_size=4 , class_weight={0:1, 1:1}, ) #, class_weight=class_weights)


Epoch 1/5
1555/1555 [==============================] - 3s 1ms/step - loss: 0.4446 - precision: 0.2581
Epoch 2/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4240 - precision: 0.0000e+00
Epoch 3/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4183 - precision: 0.5000
Epoch 4/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4124 - precision: 0.5417
Epoch 5/5
1555/1555 [==============================] - 2s 1ms/step - loss: 0.4078 - precision: 0.5385


In [15]:
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

35/35 [==============================] - 0s 899us/step
_____________________
confusion matrix: 
[[897  13]
 [178  10]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.43478260869565216


In [ ]:
"""Sprawdzę jeszcze czy istnieje możliwość stworzenia modelu w oparciu o maszynę wektorów nośnych"""

In [16]:
SVM = SVC()
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[910   0]
 [188   0]]
brak pozytywnych wyników


In [79]:
"""Wyniki dalej nie są zadowalajace, spróbuję teraz jeszcze raz z wcześniejszymi modelami, ale brakujące wartości uzupełnię przez średnią"""

In [43]:
df = pd.read_csv("df_differences.csv")
df = df.fillna(df.mean())
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)

In [ ]:
"""Random Forest"""

In [27]:
model = RandomForestClassifier(n_estimators=50, max_depth=90)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[1052    2]
 [ 197    6]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.75


In [ ]:
"""Po kilku próbach nie udało mi się zaobserwować poprawy wyników"""
"""Nasepnie sprawdzę sieć neuronową"""

In [39]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["Precision"])

model.fit(X_train, y_train, epochs=5, batch_size=4 , class_weight={0:1, 1:3}, ) #, class_weight=class_weights)


Epoch 1/5
1781/1781 [==============================] - 4s 1ms/step - loss: 0.8309 - precision: 0.2532
Epoch 2/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7994 - precision: 0.2850
Epoch 3/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7880 - precision: 0.3036
Epoch 4/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7793 - precision: 0.3222
Epoch 5/5
1781/1781 [==============================] - 2s 1ms/step - loss: 0.7682 - precision: 0.3333


In [40]:
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

40/40 [==============================] - 0s 1ms/step
_____________________
confusion matrix: 
[[855 190]
 [121  91]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.3238434163701068


In [ ]:
"""Po kilkukrotnym wyuczeniu modelu z różnymi parametramy również nie zaobserwowałem poprawy, jednak więcej razy model oznacza dane jako pozytywne, więc w przypadku użycia w aplikacji randkowej rzadziej zdarzy się sytuacja, że użytkownikowi nie zostanie zaproponowana randka, która mu odpowiada"""
"""Na koniec sprawdzę jeszcze maszynę wektorów nośnych"""

In [44]:
SVM = SVC()
SVM.fit(X_train, y_train)
y_pred = SVM.predict(X_test)
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[1056    0]
 [ 201    0]]
brak pozytywnych wyników


In [ ]:
'''Również nie zaobserwowałem znaczącej poprawy'''
'''Na koniec spróbuję jeszcze poprawić działanie modelu poprzez wyważenie danych metodą oversamplingu'''

In [59]:
df = pd.read_csv("df_differences.csv")
oversampler = SMOTE()
df = df.fillna(df.mean())
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.15)
X_train, y_train = oversampler.fit_resample(X_train, y_train)

In [52]:
model = RandomForestClassifier(n_estimators=50, max_depth=90)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[1063   14]
 [ 170   10]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.4166666666666667


In [60]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["Precision"])

model.fit(X_train, y_train, epochs=5, batch_size=4) #, class_weight=class_weights)


Epoch 1/5
2976/2976 [==============================] - 5s 1ms/step - loss: 0.5994 - precision: 0.6558
Epoch 2/5
2976/2976 [==============================] - 4s 1ms/step - loss: 0.5626 - precision: 0.6884
Epoch 3/5
2976/2976 [==============================] - 3s 1ms/step - loss: 0.5400 - precision: 0.7044
Epoch 4/5
2976/2976 [==============================] - 3s 1ms/step - loss: 0.5205 - precision: 0.7176
Epoch 5/5
2976/2976 [==============================] - 3s 1ms/step - loss: 0.5019 - precision: 0.7294


In [61]:
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

40/40 [==============================] - 0s 1ms/step
_____________________
confusion matrix: 
[[734 313]
 [112  98]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.2384428223844282


In [62]:
model = SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("_____________________")
cm = confusion_matrix(y_test, y_pred)
print("confusion matrix: ")
print(cm)
if cm[1][1]+cm[0][1] == 0:
    print("brak pozytywnych wyników")
else:
    print("prawdziwych pozytywnych w stosunku do wszystkich pozytywnych: ", cm[1][1]/(cm[1][1]+cm[0][1]))

_____________________
confusion matrix: 
[[736 311]
 [103 107]]
prawdziwych pozytywnych w stosunku do wszystkich pozytywnych:  0.25598086124401914


In [65]:
"""Ponownie nie zaobserwowałem by któych model mógł stanowić podstawę wyboru partnera"""
"""Do zastosowania w aplikacji najlepiej z przetestowanych modeli nadaje się las losowy, jednak jest on 'oszczędny'"""
"""w ilości przedstawianych rozwiązań, dla 1000 próbek zaledwie przy niektóych losowaniach podziału"""
"""na dane testowe i treningowe czasem mniej niż 10 osób zostaje ze sobą połączonych"""

'na dane testowe i treningowe czasem mniej niż 10 osób zostaje ze sobą połączonych'